# 課題2(Optional) wikipediaリンクで遊ぼう


## PageRank実装してみる - BFSの近傍探索編

### 準備1 - BFSで使用するデータの選抜


In [0]:
from google.colab import auth, drive, files, output
drive.mount('/content/drive')

In [0]:
import os
os.chdir("/content/drive/My Drive/STEP2020")

In [0]:
size = 10000 #サンプリングするページ数

In [0]:
#pagesの読み込み
f1 = open('txt/wikipedia/pages.txt')
pages = f1.read()
f1.close()
n_pages = pages.split("\n")
pages = [n_pages[i].split("\t")[1] for i in range(len(n_pages)-1)] #最後に空行が入っていたので．
del n_pages[:]

In [0]:
#page名から番号を検索できるようにしておく．
page2oldnum = dict()
for i in range(len(pages)):
    page2oldnum[pages[i]] = i
#pagename2num #確認用

In [0]:
#リンクの読み込み
f1 = open('txt/wikipedia/links.txt')
links = f1.read()
f1.close()
n_links = links.split("\n")
links = [tuple(map(int, n_links[i].split('\t'))) for i in range(len(n_links)-1)] #最後に空行が入っていたので．
del n_links[:]

In [0]:
#隣接リストの作成
direct_neighbors = [[] for i in range(len(pages))]
for link in links:
    direct_neighbors[link[0]].append(link[1])
#direct_neighbors[:2] #確認用

In [0]:
#BFSでインデックスを選ぶ
import time
from collections import deque

def bfs(direct_neighbors, appended, indexes, start, size):
    q = deque()
    q.append(start)
    appended[start] = True
    while (len(q) > 0) and (len(indexes) < size):
        current_node = q.popleft()
        indexes.append(current_node)
        for next_node in direct_neighbors[current_node]:
            if not appended[next_node]:
                q.append(next_node)
                appended[next_node] = True

start = page2oldnum['Google']
indexes = []
appended = [False for i in range(len(pages))]
bfs_time_start = time.time()
bfs(direct_neighbors, appended, indexes, start, size)
bfs_time_finish = time.time()

In [0]:
del direct_neighbors[:]

### 準備2
データを全部使うとRAMが死んでしまうので，適宜サンプリングしたデータのみを使用してメモリを節約する．

In [0]:
#BFSで選んだページとindexの対応を記録
selected_pages = []
page2num = dict()
old2new = dict()
for i in range(len(indexes)):
    selected_pages.append(pages[indexes[i]])
    page2num[pages[indexes[i]]] = i
    old2new[str(indexes[i])] = i
del pages[:]
del indexes[:]
pages = selected_pages

In [0]:
#必要なリンク以外捨てる
new_links = []
for i in range(len(links)):
    if (old2new.get(str(links[i][0]))) and (old2new.get(str(links[i][1]))): #inで検索するとO(N)の計算時間なので，辞書検索でO(1)にする．
        new_links.append(tuple([old2new[str(links[i][0])], old2new[str(links[i][1])]]))
del links[:]
links = new_links
#links[:5] #確認用

In [0]:
len(links)

907786

In [0]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        2.9G        9.3G        908K        518M         11G
Swap:            0B          0B          0B


### PageRank

In [0]:
import time
import numpy as np

In [0]:
cpu_start = time.time()
#リンク行列の定義
p_matrix = np.zeros((size, size))
e_matrix = np.ones((size, size))
PageRank = 100 * np.ones(size)
#繊維確率の計算
for i in range(len(links)):
    p_matrix[links[i][0]][links[i][1]] += 1
for i in range(size):
    if sum(p_matrix[i]) != 0:
        p_matrix[i] = p_matrix[i]/sum(p_matrix[i])
google_matrix = 0.85 * p_matrix + 0.15 * (1/size) * e_matrix
#ここからべき乗法の計算
for i in range(20):
    PageRank = np.dot(PageRank, google_matrix)
cpu_end = time.time()

print('実行時間：', cpu_end - cpu_start)

実行時間： 35.42947173118591


In [0]:
rank_index = list(reversed(sorted([[PageRank[i], i] for i in range(size)])))

In [0]:
#重要度高そうなページを10個出してみる．
for i in range(10):
    print(pages[rank_index[i][1]], rank_index[i][0])

英語 6557.477875018728
アメリカ合衆国 4645.630502081014
日本 4312.6506221205755
イギリス 2591.701070607442
フランス 2369.0268572802884
地理座標系 2139.0797574301373
ドイツ 1986.3757226396956
コンピュータ 1945.7600308616366
日本語 1890.614528098268
Portable_Document_Format 1673.2469402194054
